In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def scrape_flamecomics_chapters(url):
    """
    Scrapes chapter links and their details from a given flamecomics.xyz series URL.

    Args:
        url (str): The URL of the series on flamecomics.xyz.

    Returns:
        list: A list of dictionaries containing chapter details.
    """
    try:
        # Fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad HTTP responses
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all chapter elements
        chapters = []
        chapter_elements = soup.find_all('a', class_='ChapterCard_chapterWrapper__j8pBx')

        for chapter in chapter_elements:
            chapter_data = {
                'link': chapter['href'],  # Extract the chapter link
                'title': chapter.find('p', class_='mantine-Text-root').text.strip(),  # Extract the chapter title
                'date': chapter.find_all('p', class_='mantine-Text-root')[1].text.strip(),  # Extract the date
                'reactions': [
                    img['alt'] for img in chapter.find_all('img', class_='ReactionBarMini_reactionIcon__rkqnd')
                ]  # Extract reaction icons
            }
            chapters.append(chapter_data)

        return chapters

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Example usage
url = "https://flamecomics.xyz/series/2"
chapters = scrape_flamecomics_chapters(url)
for chapter in chapters:
    print(chapter)

Development

In [31]:

import requests
import re
import os
import json
import zipfile
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO

# url = "https://flamecomics.xyz/series/2"
url = "https://flamecomics.xyz/series/143"

response = requests.get(url)
response.raise_for_status()  # Raise an error for bad HTTP responses
soup = BeautifulSoup(response.text, 'html.parser')

# Find the <script> tag with id="__NEXT_DATA__"
script_tag = soup.find('script', id='__NEXT_DATA__', type='application/json')
if script_tag:
    # Extract the JSON content and parse it into a Python dictionary
    dat = json.loads(script_tag.string)['props']['pageProps']
else:
    print("No series object (__NEXT_DATA__) found.")

series_title = dat['series']['title']
series_title = series_title.replace(":", " -")

# Create a folder with the series name if it doesn't already exist
dir_base = os.path.join("..", "data")
if not os.path.exists(dir_base):
    os.makedirs(dir_base)
# Create a subfolder for the series
dir_series = os.path.join(dir_base, series_title)
if not os.path.exists(dir_series):
    os.makedirs(dir_series)

chapters = dat['chapters']
chapters = sorted(chapters, key=lambda x: float(x['chapter']))
for chapter in chapters:
    series_id = chapter['series_id']
    chapter_title = chapter['title']
    chapter_number = chapter['chapter']
    chapter_number = chapter_number.split(".")[0] if chapter_number.endswith(".0") else chapter_number
    chapter_token = chapter['token']
    cbz_filename = f"{series_title}, Chapter {chapter_number}.cbz"
    file_cbz = os.path.join(dir_series, cbz_filename)
    if os.path.exists(file_cbz):
        # print(f"{cbz_filename} already exists, skipping download.")
        continue
    # Check if the chapter has images
    if not chapter['images']:
        print(f"No images found for chapter {chapter_number}. Skipping download.")
        continue
    # Check if the chapter has a token
    if not chapter_token:
        print(f"No token found for chapter {chapter_number}. Skipping download.")
        continue
    # Check if the chapter has a series ID
    if not series_id:
        print(f"No series ID found for chapter {chapter_number}. Skipping download.")
        continue
    images = chapter['images']
    imgs = []
    images = dict(sorted(images.items(), key=lambda item: int(item[0])))
    print(f"Downloading: {cbz_filename}")
    for key in images:
        image = images[key]
        img_name = image['name']
        img_name = img_name.replace(" ", "%20")
        img_type = image['type']
        img_type = img_type[0] if isinstance(img_type, list) else img_type
        url_image = f"https://cdn.flamecomics.xyz/series/{series_id}/{chapter_token}/{img_name}"
        # Download the image
        image_response = requests.get(url_image)
        image_response.raise_for_status()  # Ensure the request was successful
        # Load the image into memory
        img = Image.open(BytesIO(image_response.content))
        # Add image to im memory list
        imgs.append([img, img_type])
    # Save images as a CBZ file
    with BytesIO() as cbz_buffer:
        # Save all images into the buffer as a ZIP archive
        with zipfile.ZipFile(cbz_buffer, 'w') as cbz:
            for idx, img in enumerate(imgs):
                img_filename = f"{idx + 1:03}.jpg"
                with BytesIO() as img_buffer:
                    if img[1] == 'image/jpeg':
                        img[0].save(img_buffer, format='JPEG')
                    elif img[1] == 'image/png':
                        img[0] = img[0].convert("RGB")
                        img[0].save(img_buffer, format='JPEG')
                    cbz.writestr(img_filename, img_buffer.getvalue())
        # Write the buffer to a file
        with open(file_cbz, 'wb') as cbz_file:
            cbz_file.write(cbz_buffer.getvalue())

Downloading: Solo Leveling - Ragnarok, Chapter 22.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 23.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 24.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 25.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 29.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 30.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 31.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 32.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 33.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 34.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 35.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 36.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 37.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 38.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 39.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 40.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 41.cbz
Downloading: Solo Leveling - Ragnarok, Chapter 42.cbz
Downloading: Solo Leveling -

# APIs
https://flamecomics.xyz/api/series